# BIG CATS

In [1]:
import pandas as pd
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import random

df = pd.read_csv('dataset/WILDCATS.csv')

### Prepare class indices and big cats species

In [2]:
indices = df.iloc[:, 0].unique()
species = df.iloc[:, 2].unique()
species_map = {i: big_cat_class for i, big_cat_class in zip(indices, species)}
class_ids = df[df.iloc[:, 3] == 'train'].iloc[:, 0]

### Prepare data transforms

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(224),
        transforms.RandomRotation(10),
        transforms.ToTensor()
    ]),
    'valid' : transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor()
    ]),
    'test' : transforms.Compose([
        transforms.Resize(224),
    transforms.ToTensor()
    ])
}

### Load input data to arrays with appropriate transformation

In [5]:
train_path = 'dataset/train/'
valid_path = 'dataset/valid/'
test_path = 'dataset/test/'

train_images = torchvision.datasets.ImageFolder(
    train_path, transform=data_transforms['train'])
test_images = torchvision.datasets.ImageFolder(
    test_path, transform=data_transforms['test'])
valid_images = torchvision.datasets.ImageFolder(
    valid_path, transform=data_transforms['valid'])

### Create data loaders to pass input data to neural network

In [10]:
train_loader = DataLoader(train_images, batch_size=1, shuffle=True, num_workers=0)
test_loader = DataLoader(test_images, batch_size=1, shuffle=False, num_workers=0)
valid_loader = DataLoader(valid_images, batch_size=1, shuffle=False, num_workers=0)

### Create classifier class

In [7]:
class BigCatClassifier(nn.Module):
    def __init__(self):
        super(BigCatClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)  # Warstwa konwolucyjna
        self.relu1 = nn.ReLU()  # Warstwa aktywacji ReLU
        self.flatten = nn.Flatten()  # Warstwa spłaszczająca dane
        self.fc1 = nn.Linear(16*222*222, 10)  # Warstwa w pełni połączona

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        return x
    
    id = 0


### Perform classifier training

In [16]:

model = BigCatClassifier()

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.1)

num_epochs = 20
device = torch.device("cpu")
model.to(device)

for epoch in range(num_epochs):

    model.train()

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        # Zeruj gradienty
        optimizer.zero_grad()

        # Przebieg w przód
        outputs = model(inputs)

        # Oblicz funkcję straty
        loss = criterion(outputs, targets)

        # Propagacja wsteczna
        loss.backward()

        # Aktualizacja wag
        optimizer.step()

        if (batch_idx+1) % 100 == 0:
            print(
                f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
            
    model.eval()

    # Inicjalizacja zmiennych do przechowywania wyników
    test_correct = 0
    valid_correct = 0
    test_total = 0
    valid_total = 0

    # Ocena modelu na danych testowych
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            test_total += labels.size(0)
            test_correct += predicted.eq(labels).sum().item()

    # Ocena modelu na danych walidacyjnych
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            valid_total += labels.size(0)
            valid_correct += predicted.eq(labels).sum().item()

    test_accuracy = 100 * test_correct / test_total
    valid_accuracy = 100 * valid_correct / valid_total

    print('Dokładność na danych testowych: {:.2f}%'.format(test_accuracy))
    print('Dokładność na danych walidacyjnych: {:.2f}%'.format(valid_accuracy))

    print('Trenowanie zakończone.')


Epoch [1/20], Step [100/2339], Loss: 2.7680
Epoch [1/20], Step [200/2339], Loss: 2.0546
Epoch [1/20], Step [300/2339], Loss: 1.8787
Epoch [1/20], Step [400/2339], Loss: 2.4399
Epoch [1/20], Step [500/2339], Loss: 2.6579
Epoch [1/20], Step [600/2339], Loss: 3.3131
Epoch [1/20], Step [700/2339], Loss: 2.9954
Epoch [1/20], Step [800/2339], Loss: 2.3527
Epoch [1/20], Step [900/2339], Loss: 2.0314
Epoch [1/20], Step [1000/2339], Loss: 2.6665
Epoch [1/20], Step [1100/2339], Loss: 3.0162
Epoch [1/20], Step [1200/2339], Loss: 2.2299
Epoch [1/20], Step [1300/2339], Loss: 2.5080
Epoch [1/20], Step [1400/2339], Loss: 2.4423
Epoch [1/20], Step [1500/2339], Loss: 2.9617
Epoch [1/20], Step [1600/2339], Loss: 2.7076
Epoch [1/20], Step [1700/2339], Loss: 2.4555
Epoch [1/20], Step [1800/2339], Loss: 2.7491
Epoch [1/20], Step [1900/2339], Loss: 2.1013
Epoch [1/20], Step [2000/2339], Loss: 2.5210
Epoch [1/20], Step [2100/2339], Loss: 2.5768
Epoch [1/20], Step [2200/2339], Loss: 2.2273
Epoch [1/20], Step 

### Model accuracy testing

In [13]:

model.eval()

# Inicjalizacja zmiennych do przechowywania wyników
test_correct = 0
valid_correct = 0
test_total = 0
valid_total = 0

# Ocena modelu na danych testowych
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

# Ocena modelu na danych walidacyjnych
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        valid_total += labels.size(0)
        valid_correct += predicted.eq(labels).sum().item()

test_accuracy = 100 * test_correct / test_total
valid_accuracy = 100 * valid_correct / valid_total

print('Dokładność na danych testowych: {:.2f}%'.format(test_accuracy))
print('Dokładność na danych walidacyjnych: {:.2f}%'.format(valid_accuracy))


Dokładność na danych testowych: 44.00%
Dokładność na danych walidacyjnych: 48.00%


### Save model to file

In [17]:
torch.save(model.state_dict(), 'model.pt')

AttributeError: 'BigCatClassifier' object has no attribute 'id'

### Load model from file

In [12]:

model = BigCatClassifier()
model.load_state_dict(torch.load('model.pt'))
device = torch.device("cpu")
model.to(device)


BigCatClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=788544, out_features=10, bias=True)
)